In [ ]:
RULEFIT:

In [1]:
pip install git+https://github.com/christophM/rulefit.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/christophM/rulefit.git to /tmp/pip-req-build-dhoxgd99
  Running command git clone -q https://github.com/christophM/rulefit.git /tmp/pip-req-build-dhoxgd99
  Created wheel for RuleFit: filename=RuleFit-0.3-py3-none-any.whl size=8797 sha256=898a4a60235a6515bfaf02c2026d4133cf14581e31d10611aa900e603a30ac0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-cxzmwfzz/wheels/8d/f1/8a/80d74a55f8d14e55f3f1ff050b6f9292abbb0d6de12100e410
Successfully built RuleFit


In [2]:

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
import pandas as pd 
import math
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 2.3 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from rulefit import RuleFit
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
import time


pd.set_option('display.max_rows', None)
# autopilot_r1_data = pd.read_csv("/content/autopilot_R1_6_regression_3500_value_1.csv")

# y = autopilot_r1_data.Label.values
# X = autopilot_r1_data.drop("Label", axis=1)

# features = X.columns
# X = X.values

# rf = RuleFit()
# rf.fit(X, y, feature_names=features)
# y_pred = rf.predict(X)


# rules = rf.get_rules()

# rules = rules[rules.coef != 0].sort_values(by="support")
# num_rules_rule = len(rules[rules.type == 'rule'])
# num_rules_linear = len(rules[rules.type == 'linear'])
# print(rules.sort_values('importance', ascending=False))

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from rulefit import RuleFit


boston_data = pd.read_csv("/content/tustin_R1e_regression_3500_value_1.csv")

y = boston_data.Label.values
X = boston_data.drop("Label", axis=1)
print(y)
features = X.columns
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

typ = 'classifier' #regressor or classifier

if typ == 'regressor':
    rf = RuleFit(
        rfmode='regress',
        tree_generator=RandomForestRegressor()
    )
    rf.fit(X, y, feature_names=features)
    y_pred = rf.predict(X)
    insample_rmse = np.sqrt(np.sum((y_pred - y)**2)/len(y))
    print(insample_rmse)
elif typ == 'classifier':
    y_class = y.copy()
    N = X.shape[0]
    time_stamp1 = time.time()
    rf = RuleFit(   rfmode='classify',
                    tree_generator=RandomForestClassifier()
                )
    rf.fit(X_train, y_train, feature_names=features)
    y_pred = rf.predict(X_test)
    # y_proba = rf.predict_proba(X)
    # insample_acc = sum(y_pred == y_train) / len(y_train)
    time_stamp2 = time.time()
    elapsed_time = time_stamp2 - time_stamp1
    # print(insample_acc)

rules = rf.get_rules()

rules = rules[rules.coef == 0].sort_values(by="support")
num_rules_rule = len(rules[rules.type == 'rule'])
num_rules_linear = len(rules[rules.type == 'linear'])
# print(rules.sort_values('importance', ascending=False))

In [ ]:
# rules = rf.get_rules()
# # rules = rules[rules.coef != 0].sort_values(by="support")
# rules = rules[rules.coef == 0].sort_values("support", ascending=False)

# print(rules)

In [ ]:

accuracy = accuracy_score(y_pred,y_test)

tn, fp, fn, tp = confusion_matrix(y_pred, y_test).ravel()

precisionpass = (tp) / (tp + fp)
recallpass = (tp) / (tp + fn)

precisionfail = (tn) / (tn + fn)
recallfail = (tn) / (tn + fp)

print("accuracy: ", accuracy)
print("balanced accuracy: ", balanced_accuracy_score(y_pred,y_test))
print("recall pass: ", recallpass)
print("recall fail: ", recallfail)
print("precision pass: ", precisionpass)

print("precision fail: ", precisionfail)
print("elapsed time", elapsed_time)

In [2]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 7.6 MB/s 


In [55]:
#DECISION TREE

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
import pandas as pd 
from skopt import BayesSearchCV
from skopt.space import Integer
import math

data = pd.read_csv("tustinr4_R4b_regression_3500_value_1.csv")

#y = data.Label.values
#X = data.drop("Label", axis = 1)
#X = X.drop("Fitness", axis = 1)

testdata = pd.read_csv("tustinr4_R4b_regression_3500_value_2.csv")



X_train = data.drop("Label", axis = 1)
X_train = X_train.drop("Fitness", axis = 1)
y_train = data.Label.values

X_test = testdata.drop("Label", axis = 1)
X_test = X_test.drop("Fitness", axis = 1)
y_test = testdata.Label.values

dtmodel = BayesSearchCV(tree.DecisionTreeClassifier(random_state = 0), {'min_samples_leaf': Integer(1, max(2, math.floor(len(y_train/2))))}, n_iter = 100, optimizer_kwargs = {'acq_func': 'EI'})

dtmodel.fit(X_train[:300], y_train[:300])

clf = dtmodel.best_estimator_

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

precisionpass = (tp) / (tp + fp)
recallpass = (tp) / (tp + fn)

precisionfail = (tn) / (tn + fn)
recallfail = (tn) / (tn + fp)

print("accuracy: ", accuracy)
print("balanced accuracy: ", balanced_accuracy_score(y_test, y_pred))
print("recall pass: ", recallpass)
print("recall fail: ", recallfail)
print("precision pass: ", precisionpass)

print("precision fail: ", precisionfail)

print(y_pred.tolist())

/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packa

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        52
           1       1.00      0.96      0.98        48

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100

accuracy:  0.98
balanced accuracy:  0.9791666666666667
recall pass:  0.9583333333333334
recall fail:  1.0
precision pass:  1.0
precision fail:  0.9629629629629629
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]


In [14]:
X_train

,Type,TrainDelta,TestDelta,AP_Eng1,AP_Eng2,AP_Eng3,HDG_Mode1,HDG_Mode2,HDG_Mode3,ALT_Mode1,...,HDG_Ref3,TurnK1,TurnK2,TurnK3,ALT_Ref1,ALT_Ref2,ALT_Ref3,Pwheel1,Pwheel2,Pwheel3
0,0,0,0,1,0,0,1,1,0,1,...,-89.042279,32.909379,43.025749,22.206740,497.009601,537.967562,236.358939,20.737285,21.548637,28.651301
1,0,0,0,0,1,0,1,0,1,0,...,-93.109998,19.364039,38.210679,31.119822,195.527223,790.138195,288.295313,29.198833,15.097425,-15.770147
2,0,0,0,0,1,0,0,1,1,0,...,-113.480226,2.899508,25.331430,28.427521,519.701619,89.117520,592.265014,17.260841,-26.779056,-21.814282
3,0,0,0,1,1,1,0,1,1,0,...,121.834662,4.658832,8.634422,10.661547,587.324370,334.795811,820.148106,22.978466,-4.269740,11.779975
4,0,0,0,1,0,1,0,0,0,1,...,-170.518963,13.341847,36.440570,7.769409,30.223196,987.527111,477.129879,19.703314,14.632122,18.244655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0,0,0,1,0,0,1,1,1,1,...,124.395699,5.861944,15.780894,5.088956,282.254177,766.216734,258.725139,17.198286,-7.031181,17.223219
597,0,0,0,1,0,0,0,0,1,1,...,-160.581224,29.354587,1.599090,35.123387,800.578315,347.872033,463.557716,-18.944399,-12.456607,10.819100
598,0,0,0,0,0,0,0,1,0,1,...,178.456659,15.358528,43.492596,17.467946,244.007155,464.659339,332.364667,14.363487,-25.310533,25.517936
599,0,0,0,1,0,1,0,1,0,1,...,179.430905,22.110760,17.269822,43.897891,487.446337,775.391812,18.176364,-3.547910,20.172602,-13.130236


In [32]:
#code to extract some rules - Can also be done manually

from sklearn.tree import _tree
from sklearn import tree
import numpy as np
import pandas as pd
from skopt import BayesSearchCV
from skopt.space import Integer
import math

def get_rules(tree, feature_names, class_names): #this function is taken from: https://mljar.com/blog/extract-rules-decision-tree/
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules


data = pd.read_csv("autopilot_R1_4_1_regression_3500_value_1.csv")

X_train = data.drop("Label", axis = 1)
X_train = X_train.drop("Fitness", axis = 1)
y_train = data.Label.values

dtmodel = BayesSearchCV(tree.DecisionTreeClassifier(random_state = 0), {'min_samples_leaf': Integer(1, max(2, math.floor(len(y_train/2))))}, n_iter = 100, optimizer_kwargs = {'acq_func': 'EI'})

dtmodel.fit(X_train[:300], y_train[:300])

clf = dtmodel.best_estimator_

clf.fit(X_train, y_train)

c = 0

rules = get_rules(clf, X_train.columns, ['0','1'])
for r in rules:    
  if r.find('class: 0') != -1 and r.find('100.0') != -1:
    print(r)
    c = c + 1

/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packa

if (HDG_Mode1 > 0.5) and (TurnK1 > 2.82) and (TurnK2 > 2.061) and (ALT_Ref1 > 9.864) and (Pwheel1 > -28.363) then class: 0 (proba: 100.0%) | based on 231 samples
if (HDG_Mode1 <= 0.5) and (AP_Eng1 <= 0.5) and (TurnK1 > 3.041) then class: 0 (proba: 100.0%) | based on 128 samples
if (HDG_Mode1 <= 0.5) and (AP_Eng1 > 0.5) and (TurnK1 > 30.219) then class: 0 (proba: 100.0%) | based on 53 samples
if (HDG_Mode1 > 0.5) and (TurnK1 <= 2.82) and (AP_Eng1 > 0.5) then class: 0 (proba: 100.0%) | based on 13 samples
if (HDG_Mode1 > 0.5) and (TurnK1 > 2.82) and (TurnK2 <= 2.061) and (Pwheel3 > -10.455) then class: 0 (proba: 100.0%) | based on 9 samples
if (HDG_Mode1 > 0.5) and (TurnK1 > 2.82) and (TurnK2 > 2.061) and (ALT_Ref1 > 9.864) and (Pwheel1 <= -28.363) and (Pwheel1 <= -28.426) then class: 0 (proba: 100.0%) | based on 7 samples
if (HDG_Mode1 > 0.5) and (TurnK1 > 2.82) and (TurnK2 > 2.061) and (ALT_Ref1 <= 9.864) and (ALT_Ref1 <= 7.313) then class: 0 (proba: 100.0%) | based on 4 samples
if (HD

In [34]:
!pip install wittgenstein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 124 kB 11.9 MB/s 


In [56]:
#Decision Rules

import wittgenstein as lw
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from skopt import BayesSearchCV
from skopt.space import Integer

data = pd.read_csv("tustinr4_R4b_regression_3500_value_1.csv")

#y = data.Label.values
#X = data.drop("Label", axis = 1)
#X = X.drop("Fitness", axis = 1)

testdata = pd.read_csv("tustinr4_R4b_regression_3500_value_2.csv")



X_train = data.drop("Label", axis = 1)
X_train = X_train.drop("Fitness", axis = 1)
y_train = data.Label.values

X_test = testdata.drop("Label", axis = 1)
X_test = X_test.drop("Fitness", axis = 1)
y_test = testdata.Label.values


drmodel = BayesSearchCV(lw.RIPPER(random_state = 0), {'prune_size': [0.1, 0.8], 'k': [1, 3]}, n_iter = 100, optimizer_kwargs = {'acq_func': 'EI'})

drmodel.fit(X_train[:300], y_train[:300])

clf = drmodel.best_estimator_

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

precisionpass = (tp) / (tp + fp)
recallpass = (tp) / (tp + fn)

precisionfail = (tn) / (tn + fn)
recallfail = (tn) / (tn + fp)

print("accuracy: ", accuracy)
print("recall pass: ", recallpass)
print("recall fail: ", recallfail)
print("precision pass: ", precisionpass)

print("precision fail: ", precisionfail)


              precision    recall  f1-score   support

           0       0.86      0.96      0.91        52
           1       0.95      0.83      0.89        48

    accuracy                           0.90       100
   macro avg       0.91      0.90      0.90       100
weighted avg       0.91      0.90      0.90       100

accuracy:  0.9
recall pass:  0.8333333333333334
recall fail:  0.9615384615384616
precision pass:  0.9523809523809523
precision fail:  0.8620689655172413


In [63]:
import wittgenstein as lw
#extract rules for ensemble DR
from operator import itemgetter
import pandas as pd

def GetConfSamp(dictt, indexx, feat):

  data = pd.read_csv("tustinr4_R4b_regression_3500_value_1.csv")
  

  trues = 0
  inrange = 0

  for i in range(len(data.index)):

    count = 0

    #print(dictt)
    for var in dictt:

      if dictt[var][2] == "between":

        if  dictt[var][0]<= data.loc[i, var]<= dictt[var][1]:
          count = count + 1
      
      elif dictt[var][2] == "greater":

        if  data.loc[i, var]>= dictt[var][0]:
          count = count + 1

      elif dictt[var][2] == "smaller":

        if  data.loc[i, var]<= dictt[var][0]:
          count = count + 1 

      elif dictt[var][2] == "equal":
        
        if  data.loc[i, var]== dictt[var][0]:
          count = count + 1 
        
    if count == len(dictt): #all the variables are in the ranges
      inrange = inrange + 1
      if data.loc[i, 'Label'] == 0: #class 0
          trues = trues + 1
    else:
      continue
  
  #print(trues, inrange)
  return (trues/inrange)*100, inrange


def ExtractBounds(rule):
  #print(type(rule))
  #print("rule", rule)
  rule = rule[1:-1].split('^')
  #print("rule", rule)

  dictt = {}

  for i in range(len(rule)):
    #print(rule[i])
    if rule[i].find('>') != -1:

      dictt[rule[i][:rule[i].find('=')]] = [float(rule[i][rule[i].find('=')+2 : len(rule[i])]), "", "greater"]

    elif rule[i].find('<') != -1:

      dictt[rule[i][:rule[i].find('=')]] = [float(rule[i][rule[i].find('<')+1 : len(rule[i])]), "", "smaller"]

    elif rule[i].find('-') != -1:

      for k in range(rule[i].find('=')+2, len(rule[i])):
        if rule[i][k] == '-':
          a = rule[i][rule[i].find('=')+1: k]
          k = k
          break
      #print(a, "****", k+1, rule[i][k+1:len(rule[i])])
      dictt[rule[i][:rule[i].find('=')]] = [float(a), float(rule[i][k+1:len(rule[i])]), "between"]

    elif rule[i].find('=') != -1:

      #print(rule[i][rule[i].find('=')+1 : len(rule[i])])
      #print(len(rule[i][rule[i].find('=')+1 : len(rule[i])]))
      dictt[rule[i][:rule[i].find('=')]] = [float(rule[i][rule[i].find('=')+1 : len(rule[i])]), "", "equal"]      

  #print("dictt is", dictt)
  
  return dictt



rules= []


for i in range(1, 2):

  data = pd.read_csv("tustinr4_R4b_regression_3500_value_1.csv") 
  

  X_train = data.drop("Label", axis = 1)
  X_train = X_train.drop("Fitness", axis = 1)
  y_train = data.Label.values

  drmodel = BayesSearchCV(lw.RIPPER(random_state = 0), {'prune_size': [0.1, 0.8], 'k': [1, 3]}, n_iter = 100, optimizer_kwargs = {'acq_func': 'EI'})

  drmodel.fit(X_train[:300], y_train[:300])

  clf = drmodel.best_estimator_

  try:
    clf.fit(X_train, y_train, pos_class=0) #to get the fail class rules
  except:
    print("No rules")
    print('*****************')
    continue
  #clf.out_model()
  clf.ruleset_.out_pretty()

  #print(clf.ruleset_.count_conds())
  rules.append(clf.ruleset_)


  print("******************")


#print(rules)

newrules = []

res = pd.DataFrame(columns= ['Run'+str(i) for i in range(1,21)])

confs = [[] for i in range(20)]

for i in range(len(rules)):
  listt = []
  for j in range(len(rules[i])):

    listt.append(str(rules[i][j]))

    c = str(rules[i][j])
    dictt = ExtractBounds(c)

    #print(a,b)
  
    conf, samples = GetConfSamp(dictt, i, X_train.columns)

    confs[i].append((conf, samples))

    print(conf, samples)

  
  print("*************")
  newrules.append(listt)

res.loc[res.shape[0]] = [confs[k] for k in range(20)]
res.to_excel('results.xlsx')



[[BL=0.11-2.14 ^ TL=3.64-5.75] V
[BL=-1.84-0.11] V
[BL=0.11-2.14 ^ TL=-4.2--2.69] V
[BL=0.11-2.14 ^ TL=-6.09--4.2] V
[BL=0.11-2.14] V
[BL=5.99-8.0 ^ TL=-8.14--6.09] V
[BL=-4.12--1.84 ^ TL=5.75-7.87] V
[TL=<-8.14 ^ BL=>8.0] V
[BL=5.99-8.0 ^ TL=-4.2--2.69] V
[BL=<-8.06 ^ TL=5.75-7.87] V
[TL=-6.09--4.2 ^ BL=3.86-5.99] V
[TL=-0.49-1.44 ^ BL=-8.06--5.96] V
[BL=>8.0 ^ TL=-6.09--4.2] V
[TL=<-8.14 ^ BL=3.86-5.99] V
[TL=-2.69--0.49 ^ BL=5.99-8.0] V
[BL=-4.12--1.84 ^ TL=3.64-5.75] V
[IC=>16.18 ^ BL=-8.06--5.96] V
[TL=-2.69--0.49 ^ BL=2.14-3.86] V
[TL=-2.69--0.49 ^ BL=>8.0] V
[BL=2.14-3.86 ^ TL=-4.2--2.69] V
[TL=<-8.14 ^ BL=2.14-3.86] V
[BL=<-8.06 ^ TL=3.64-5.75] V
[TL=>7.87 ^ BL=-4.12--1.84] V
[BL=-5.96--4.12 ^ TL=1.44-3.64] V
[TL=<-8.14 ^ BL=5.99-8.0] V
[BL=-8.06--5.96 ^ TL=5.75-7.87] V
[TL=-0.49-1.44 ^ BL=<-8.06] V
[BL=-8.06--5.96 ^ TL=>7.87] V
[TL=-8.14--6.09 ^ BL=2.14-3.86] V
[BL=>8.0 ^ TL=-4.2--2.69] V
[BL=3.86-5.99 ^ TL=-8.14--6.09] V
[TL=-8.14--6.09 ^ BL=>8.0] V
[TL=>7.87 ^ BL=-5.96--4.12